In [25]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
!pip install ultralytics unidecode

In [14]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

# Đường dẫn đến thư mục data_test
data_dir = '/content/drive/MyDrive/object_detection/data_test'

datagen = ImageDataGenerator(rescale=1./255)

# Tạo generator cho dữ liệu kiểm tra
test_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),  # Kích thước của ảnh đầu vào (thay đổi nếu cần thiết)
    batch_size=32,
    class_mode='categorical',
    shuffle=False  # Không xáo trộn để đảm bảo thứ tự của nhãn
)

# Danh sách nhãn của mô hình CNN
# ['banh-hoi-heo-quay', 'bun-bo-hue', 'bun-ca', 'bun-dau-mam-tom', 'bun-hai-san-be-be', 'bun-moc', 'bun-muc',
# 'bun-nuoc-leo', 'cao-lau', 'com-ga', 'com-tam-long-xuyen', 'hu-tieu-my-tho', 'mi-quang', 'pho-ha-noi']
labels_cnn = list(test_generator.class_indices.keys())
class_names = list(test_generator.class_indices.keys())
print(labels_cnn)

labels_d2d = ['bun-ca', 'hu-tieu-my-tho', 'bun-nuoc-leo', 'com-tam-long-xuyen', 'bun-hai-san-be-be', 'banh-hoi-heo-quay',
              'com-ga', 'cao-lau', 'mi-quang', 'bun-bo-hue', 'pho-ha-noi', 'bun-muc', 'bun-moc', 'bun-dau-mam-tom']

Found 380 images belonging to 14 classes.
['banh-hoi-heo-quay', 'bun-bo-hue', 'bun-ca', 'bun-dau-mam-tom', 'bun-hai-san-be-be', 'bun-moc', 'bun-muc', 'bun-nuoc-leo', 'cao-lau', 'com-ga', 'com-tam-long-xuyen', 'hu-tieu-my-tho', 'mi-quang', 'pho-ha-noi']


In [15]:
class_indices = test_generator.class_indices

# Tạo dictionary để ánh xạ từ tên nhãn sang số
label_to_index = {name: index for index, name in class_indices.items()}

print(label_to_index)

{0: 'banh-hoi-heo-quay', 1: 'bun-bo-hue', 2: 'bun-ca', 3: 'bun-dau-mam-tom', 4: 'bun-hai-san-be-be', 5: 'bun-moc', 6: 'bun-muc', 7: 'bun-nuoc-leo', 8: 'cao-lau', 9: 'com-ga', 10: 'com-tam-long-xuyen', 11: 'hu-tieu-my-tho', 12: 'mi-quang', 13: 'pho-ha-noi'}


In [7]:
import pandas as pd
from ultralytics import YOLO, RTDETR

excel_file_path = '/content/drive/MyDrive/object_detection/rules.xlsx'
classes_path = '/content/drive/MyDrive/object_detection/classes.txt'

df = pd.read_excel(excel_file_path)
classes_ingre = []

with open(classes_path, 'r', encoding='utf-8') as file:
    lines = [line.strip() for line in file]
    classes_ingre = lines

model_yolov8 = YOLO('/content/drive/MyDrive/object_detection/models/yolov8_best_862.pt')
model_yolov9 = YOLO('/content/drive/MyDrive/object_detection/models/yolov9_best_85.pt')
model_rtdetr = RTDETR('/content/drive/MyDrive/object_detection/models/rt-detr_best_865.pt')

In [8]:
from unidecode import unidecode

def preprocess_dataframe():
    global df
    normalized_df = df.drop(df.columns[[0, -1]], axis=1)

    for index, row in normalized_df.iterrows():
        for col in normalized_df.columns:
            value = row[col]
            if pd.notna(value):
                name_without_accents = unidecode(value)
                values = name_without_accents.split(', ')
                for i in range(len(values)):
                    values[i] = values[i].replace(' ', '-')
                row[col] = ', '.join(values)

    return normalized_df

normalized_df = preprocess_dataframe()

In [9]:
def detect_ingredients(model, img):
    results = model(img, conf=0.3, save=False)
    names = results[0].names
    detected_cls = results[0].boxes.cls.tolist()
    boxes = results[0].boxes.xyxy.tolist()
    return names, detected_cls, boxes

In [10]:
def find_foodname_ver_narrow(set_detected_ingre):
    global normalized_df
    global df
    global food_names
    distances = []

    for index, row in normalized_df.iterrows():
        values = {}
        values["prior"] = set(row["Priorities"].split(', '))
        values["main"] = set(row["Ingredients"].split(', '))
        values["extra"] = set(row["Secondary Ingredients"].split(', ')) if pd.notna(row["Secondary Ingredients"]) else set()

        # Không khớp với ƯU TIÊN
        no_match_prior = values["prior"].difference(set_detected_ingre)

        # Không khớp với CHÍNH
        no_match_main = values["main"].difference(set_detected_ingre)

        # Khớp với PHỤ
        match_extra = values["extra"].intersection(set_detected_ingre)

        # Phần tử thuộc detect nhưng không thuộc bộ luật CHÍNH & PHỤ
        combine = values["prior"].union(values["main"])
        combine = combine.union(values["extra"])
        redundancy = set_detected_ingre.difference(combine)

        shortage_score = (len(no_match_prior) * 10) + len(no_match_main) - (len(match_extra)*0.5)
        redundancy_score = len(redundancy)
        score = shortage_score + redundancy_score

        # print(index, score)
        distances.append(score)

    min_value = min(distances)
    min_indices = [i for i, value in enumerate(distances) if value == min_value]

    predicted_food = []
    for min_index in min_indices:
        predicted_food.append(labels_d2d[min_index])

    return distances, predicted_food[0]

In [11]:
def convertIdx2Class(set_index, classes):
    result_set = {classes[item] for item in set_index}
    return result_set

In [12]:
import cv2

def recognize_food(detector, file_predict):
    global classes_ingre

    img = cv2.imread(file_predict)
    detector_names, detector_detected_cls, detector_boxes = detect_ingredients(detector, img)

    detector_rm_duplicates = set(detector_detected_cls)
    detector_rm_duplicates = [int(number) for number in detector_rm_duplicates]
    detector_detected_cls = list(detector_rm_duplicates)
    detector_rm_duplicates = convertIdx2Class(detector_rm_duplicates, classes_ingre)

    detector_distances, detector_predicted_food = find_foodname_ver_narrow(detector_rm_duplicates)

    return detector_predicted_food

# Kết quả Yolov8 + D2D

In [16]:
file_paths = test_generator.filepaths

predY = []

for index, img_predict in enumerate(file_paths):
    print(index)
    food = recognize_food(model_yolov8, img_predict)
    # Chuyển đổi từ tên nhãn sang số tương ứng
    predicted_index = next((index for index, label in label_to_index.items() if label == food), None)
    predY.append(predicted_index)
    print("------")

0

0: 544x640 8 thit-heo-quays, 8 banh-hois, 1 dua-chua, 88.9ms
Speed: 5.2ms preprocess, 88.9ms inference, 2.2ms postprocess per image at shape (1, 3, 544, 640)
------
1

0: 640x576 3 thit-heo-quays, 6 banh-hois, 1 dua-chua, 152.6ms
Speed: 3.0ms preprocess, 152.6ms inference, 2.2ms postprocess per image at shape (1, 3, 640, 576)
------
2

0: 608x640 6 thit-heo-quays, 3 banh-hois, 1 dua-chua, 171.0ms
Speed: 4.8ms preprocess, 171.0ms inference, 1.3ms postprocess per image at shape (1, 3, 608, 640)
------
3

0: 384x640 7 thit-heo-quays, 1 banh-hoi, 77.5ms
Speed: 4.4ms preprocess, 77.5ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)
------
4

0: 512x640 5 thit-heo-quays, 8 banh-hois, 78.9ms
Speed: 4.3ms preprocess, 78.9ms inference, 1.2ms postprocess per image at shape (1, 3, 512, 640)
------
5

0: 512x640 16 thit-heo-quays, 19 banh-hois, 74.0ms
Speed: 4.8ms preprocess, 74.0ms inference, 1.5ms postprocess per image at shape (1, 3, 512, 640)
------
6

0: 640x480 5 thit-he

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,



0: 384x640 4 thit-heo-quays, 5 banh-hois, 62.7ms
Speed: 3.0ms preprocess, 62.7ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)
------
8

0: 544x640 10 thit-heo-quays, 4 banh-hois, 89.5ms
Speed: 3.7ms preprocess, 89.5ms inference, 1.3ms postprocess per image at shape (1, 3, 544, 640)
------
9

0: 480x640 3 thit-heo-quays, 2 banh-hois, 74.9ms
Speed: 3.1ms preprocess, 74.9ms inference, 1.2ms postprocess per image at shape (1, 3, 480, 640)
------
10

0: 640x640 10 thit-heo-quays, 7 banh-hois, 88.0ms
Speed: 5.5ms preprocess, 88.0ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 640)
------
11

0: 640x640 10 thit-heo-quays, 7 banh-hois, 87.5ms
Speed: 6.4ms preprocess, 87.5ms inference, 1.6ms postprocess per image at shape (1, 3, 640, 640)
------
12

0: 640x480 4 thit-heo-quays, 3 banh-hois, 62.6ms
Speed: 4.6ms preprocess, 62.6ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 480)
------
13

0: 544x640 6 banh-hois, 1 dua-chua, 3 cha-gios, 74.9ms
S

In [17]:
from sklearn.metrics import confusion_matrix

# Lấy nhãn thực tế từ generator
true_classes = test_generator.classes

# Tính các chỉ số đánh giá
accuracy = accuracy_score(true_classes, predY)
precision = precision_score(true_classes, predY, average='weighted')
recall = recall_score(true_classes, predY, average='weighted')
f1 = f1_score(true_classes, predY, average='weighted')

# In ra kết quả
print('Accuracy:', accuracy * 100)
print('Precision:', precision * 100)
print('Recall:', recall * 100)
print('F1 Score:', f1* 100)

# In báo cáo chi tiết
print('\nClassification Report:')
print(classification_report(true_classes, predY, target_names=class_names))

# Tính accuracy cho từng nhãn
conf_matrix = confusion_matrix(true_classes, predY)
class_accuracies = conf_matrix.diagonal() / conf_matrix.sum(axis=1)

print('\nAccuracy for each class:')
for class_name, class_accuracy in zip(class_names, class_accuracies):
    print(f'{class_name}: {class_accuracy:.2f}')

Accuracy: 94.21052631578948
Precision: 95.36190779367773
Recall: 94.21052631578948
F1 Score: 94.34501455419044

Classification Report:
                    precision    recall  f1-score   support

 banh-hoi-heo-quay       0.94      1.00      0.97        15
        bun-bo-hue       0.97      0.97      0.97        36
            bun-ca       0.65      1.00      0.78        20
   bun-dau-mam-tom       1.00      0.92      0.96        25
 bun-hai-san-be-be       1.00      0.93      0.97        30
           bun-moc       0.95      1.00      0.97        19
           bun-muc       0.92      0.92      0.92        13
      bun-nuoc-leo       1.00      0.77      0.87        26
           cao-lau       0.97      0.92      0.94        37
            com-ga       1.00      1.00      1.00        40
com-tam-long-xuyen       1.00      1.00      1.00        36
    hu-tieu-my-tho       1.00      0.71      0.83        14
          mi-quang       0.86      0.94      0.90        33
        pho-ha-noi      

# Kết quả Yolov9 + D2D

In [ ]:
file_paths = test_generator.filepaths

predY = []

for index, img_predict in enumerate(file_paths):
    print(index)
    food = recognize_food(model_yolov9, img_predict)
    # Chuyển đổi từ tên nhãn sang số tương ứng
    predicted_index = next((index for index, label in label_to_index.items() if label == food), None)
    predY.append(predicted_index)
    print("------")

In [19]:
from sklearn.metrics import confusion_matrix

# Lấy nhãn thực tế từ generator
true_classes = test_generator.classes

# Tính các chỉ số đánh giá
accuracy = accuracy_score(true_classes, predY)
precision = precision_score(true_classes, predY, average='weighted')
recall = recall_score(true_classes, predY, average='weighted')
f1 = f1_score(true_classes, predY, average='weighted')

# In ra kết quả
print('Accuracy:', accuracy * 100)
print('Precision:', precision * 100)
print('Recall:', recall * 100)
print('F1 Score:', f1* 100)

# In báo cáo chi tiết
print('\nClassification Report:')
print(classification_report(true_classes, predY, target_names=class_names))

# Tính accuracy cho từng nhãn
conf_matrix = confusion_matrix(true_classes, predY)
class_accuracies = conf_matrix.diagonal() / conf_matrix.sum(axis=1)

print('\nAccuracy for each class:')
for class_name, class_accuracy in zip(class_names, class_accuracies):
    print(f'{class_name}: {class_accuracy:.2f}')

Accuracy: 94.73684210526315
Precision: 95.6808044953582
Recall: 94.73684210526315
F1 Score: 94.80000812820457

Classification Report:
                    precision    recall  f1-score   support

 banh-hoi-heo-quay       0.94      1.00      0.97        15
        bun-bo-hue       0.95      1.00      0.97        36
            bun-ca       0.69      1.00      0.82        20
   bun-dau-mam-tom       1.00      0.96      0.98        25
 bun-hai-san-be-be       1.00      0.93      0.97        30
           bun-moc       1.00      1.00      1.00        19
           bun-muc       0.87      1.00      0.93        13
      bun-nuoc-leo       1.00      0.73      0.84        26
           cao-lau       0.97      0.92      0.94        37
            com-ga       1.00      1.00      1.00        40
com-tam-long-xuyen       1.00      1.00      1.00        36
    hu-tieu-my-tho       1.00      0.79      0.88        14
          mi-quang       0.86      0.94      0.90        33
        pho-ha-noi       

# Kết quả RT-DETR + D2D

In [ ]:
file_paths = test_generator.filepaths

predY = []

for index, img_predict in enumerate(file_paths):
    food = recognize_food(model_rtdetr, img_predict)
    # Chuyển đổi từ tên nhãn sang số tương ứng
    predicted_index = next((index for index, label in label_to_index.items() if label == food), None)
    predY.append(predicted_index)

In [22]:
from sklearn.metrics import confusion_matrix

# Lấy nhãn thực tế từ generator
true_classes = test_generator.classes

# Tính các chỉ số đánh giá
accuracy = accuracy_score(true_classes, predY)
precision = precision_score(true_classes, predY, average='weighted')
recall = recall_score(true_classes, predY, average='weighted')
f1 = f1_score(true_classes, predY, average='weighted')

# In ra kết quả
print('Accuracy:', accuracy * 100)
print('Precision:', precision * 100)
print('Recall:', recall * 100)
print('F1 Score:', f1* 100)

# In báo cáo chi tiết
print('\nClassification Report:')
print(classification_report(true_classes, predY, target_names=class_names))

# Tính accuracy cho từng nhãn
conf_matrix = confusion_matrix(true_classes, predY)
class_accuracies = conf_matrix.diagonal() / conf_matrix.sum(axis=1)

print('\nAccuracy for each class:')
for class_name, class_accuracy in zip(class_names, class_accuracies):
    print(f'{class_name}: {class_accuracy:.2f}')

Accuracy: 94.73684210526315
Precision: 95.93324181566966
Recall: 94.73684210526315
F1 Score: 94.97862532756365

Classification Report:
                    precision    recall  f1-score   support

 banh-hoi-heo-quay       1.00      1.00      1.00        15
        bun-bo-hue       1.00      0.94      0.97        36
            bun-ca       0.65      1.00      0.78        20
   bun-dau-mam-tom       1.00      0.92      0.96        25
 bun-hai-san-be-be       1.00      0.93      0.97        30
           bun-moc       0.95      1.00      0.97        19
           bun-muc       1.00      0.92      0.96        13
      bun-nuoc-leo       1.00      0.81      0.89        26
           cao-lau       0.94      0.92      0.93        37
            com-ga       1.00      1.00      1.00        40
com-tam-long-xuyen       1.00      0.97      0.99        36
    hu-tieu-my-tho       1.00      0.86      0.92        14
          mi-quang       0.84      0.94      0.89        33
        pho-ha-noi      

# Kết quả MobileNet

In [26]:
from sklearn.metrics import confusion_matrix

model = tf.keras.models.load_model('/content/drive/MyDrive/object_detection/models/mobilenet.h5')

predictions = model.predict(test_generator)
predicted_classes = np.argmax(predictions, axis=1)

# Lấy nhãn thực tế từ generator
true_classes = test_generator.classes

# Tính các chỉ số đánh giá
accuracy = accuracy_score(true_classes, predicted_classes)
precision = precision_score(true_classes, predicted_classes, average='weighted')
recall = recall_score(true_classes, predicted_classes, average='weighted')
f1 = f1_score(true_classes, predicted_classes, average='weighted')

# In ra kết quả
print('Accuracy:', accuracy * 100)
print('Precision:', precision * 100)
print('Recall:', recall * 100)
print('F1 Score:', f1* 100)

# In báo cáo chi tiết
print('\nClassification Report:')
print(classification_report(true_classes, predicted_classes, target_names=class_names))

# Tính accuracy cho từng nhãn
conf_matrix = confusion_matrix(true_classes, predicted_classes)
class_accuracies = conf_matrix.diagonal() / conf_matrix.sum(axis=1)

print('\nAccuracy for each class:')
for class_name, class_accuracy in zip(class_names, class_accuracies):
    print(f'{class_name}: {class_accuracy:.2f}')

12/12 [==============================] - 6s 288ms/step
Accuracy: 63.68421052631579
Precision: 73.88131883984464
Recall: 63.68421052631579
F1 Score: 60.08327421431432

Classification Report:
                    precision    recall  f1-score   support

 banh-hoi-heo-quay       1.00      0.73      0.85        15
        bun-bo-hue       1.00      0.03      0.05        36
            bun-ca       0.43      0.50      0.47        20
   bun-dau-mam-tom       0.68      1.00      0.81        25
 bun-hai-san-be-be       0.92      0.37      0.52        30
           bun-moc       0.93      0.68      0.79        19
           bun-muc       0.80      0.31      0.44        13
      bun-nuoc-leo       0.93      0.50      0.65        26
           cao-lau       0.48      0.92      0.63        37
            com-ga       0.75      0.82      0.79        40
com-tam-long-xuyen       0.90      0.72      0.80        36
    hu-tieu-my-tho       0.67      0.14      0.24        14
          mi-quang       0.53

# Kết quả InceptionV3

In [27]:
from sklearn.metrics import confusion_matrix

model = tf.keras.models.load_model('/content/drive/MyDrive/object_detection/models/inceptionv3.h5')

predictions = model.predict(test_generator)
predicted_classes = np.argmax(predictions, axis=1)

# Lấy nhãn thực tế từ generator
true_classes = test_generator.classes

# Tính các chỉ số đánh giá
accuracy = accuracy_score(true_classes, predicted_classes)
precision = precision_score(true_classes, predicted_classes, average='weighted')
recall = recall_score(true_classes, predicted_classes, average='weighted')
f1 = f1_score(true_classes, predicted_classes, average='weighted')

# In ra kết quả
print('Accuracy:', accuracy * 100)
print('Precision:', precision * 100)
print('Recall:', recall * 100)
print('F1 Score:', f1* 100)

# In báo cáo chi tiết
print('\nClassification Report:')
print(classification_report(true_classes, predicted_classes, target_names=class_names))

# Tính accuracy cho từng nhãn
conf_matrix = confusion_matrix(true_classes, predicted_classes)
class_accuracies = conf_matrix.diagonal() / conf_matrix.sum(axis=1)

print('\nAccuracy for each class:')
for class_name, class_accuracy in zip(class_names, class_accuracies):
    print(f'{class_name}: {class_accuracy:.2f}')

12/12 [==============================] - 15s 805ms/step
Accuracy: 52.10526315789473
Precision: 60.0072244207116
Recall: 52.10526315789473
F1 Score: 51.16937473867017

Classification Report:
                    precision    recall  f1-score   support

 banh-hoi-heo-quay       0.50      0.73      0.59        15
        bun-bo-hue       0.85      0.31      0.45        36
            bun-ca       0.28      0.40      0.33        20
   bun-dau-mam-tom       0.87      0.52      0.65        25
 bun-hai-san-be-be       0.67      0.53      0.59        30
           bun-moc       0.41      0.89      0.57        19
           bun-muc       0.24      0.54      0.33        13
      bun-nuoc-leo       0.85      0.42      0.56        26
           cao-lau       0.70      0.43      0.53        37
            com-ga       0.46      0.82      0.59        40
com-tam-long-xuyen       0.76      0.72      0.74        36
    hu-tieu-my-tho       0.00      0.00      0.00        14
          mi-quang       0.73

# Kết quả DenseNet201

In [28]:
from sklearn.metrics import confusion_matrix

model = tf.keras.models.load_model('/content/drive/MyDrive/object_detection/models/densenet201.h5')

predictions = model.predict(test_generator)
predicted_classes = np.argmax(predictions, axis=1)

# Lấy nhãn thực tế từ generator
true_classes = test_generator.classes

# Tính các chỉ số đánh giá
accuracy = accuracy_score(true_classes, predicted_classes)
precision = precision_score(true_classes, predicted_classes, average='weighted')
recall = recall_score(true_classes, predicted_classes, average='weighted')
f1 = f1_score(true_classes, predicted_classes, average='weighted')

# In ra kết quả
print('Accuracy:', accuracy * 100)
print('Precision:', precision * 100)
print('Recall:', recall * 100)
print('F1 Score:', f1* 100)

# In báo cáo chi tiết
print('\nClassification Report:')
print(classification_report(true_classes, predicted_classes, target_names=class_names))

# Tính accuracy cho từng nhãn
conf_matrix = confusion_matrix(true_classes, predicted_classes)
class_accuracies = conf_matrix.diagonal() / conf_matrix.sum(axis=1)

print('\nAccuracy for each class:')
for class_name, class_accuracy in zip(class_names, class_accuracies):
    print(f'{class_name}: {class_accuracy:.2f}')

12/12 [==============================] - 22s 917ms/step
Accuracy: 52.63157894736842
Precision: 73.21399868212214
Recall: 52.63157894736842
F1 Score: 47.302560017313446

Classification Report:
                    precision    recall  f1-score   support

 banh-hoi-heo-quay       1.00      0.67      0.80        15
        bun-bo-hue       1.00      0.08      0.15        36
            bun-ca       1.00      0.10      0.18        20
   bun-dau-mam-tom       0.86      1.00      0.93        25
 bun-hai-san-be-be       1.00      0.10      0.18        30
           bun-moc       0.25      0.84      0.38        19
           bun-muc       0.45      0.77      0.57        13
      bun-nuoc-leo       1.00      0.15      0.27        26
           cao-lau       0.64      0.86      0.74        37
            com-ga       0.71      0.68      0.69        40
com-tam-long-xuyen       0.61      0.75      0.67        36
    hu-tieu-my-tho       0.00      0.00      0.00        14
          mi-quang       1.

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:13

# Kết quả VGG16

In [29]:
from sklearn.metrics import confusion_matrix

model = tf.keras.models.load_model('/content/drive/MyDrive/object_detection/models/vgg16.h5')

predictions = model.predict(test_generator)
predicted_classes = np.argmax(predictions, axis=1)

# Lấy nhãn thực tế từ generator
true_classes = test_generator.classes

# Tính các chỉ số đánh giá
accuracy = accuracy_score(true_classes, predicted_classes)
precision = precision_score(true_classes, predicted_classes, average='weighted')
recall = recall_score(true_classes, predicted_classes, average='weighted')
f1 = f1_score(true_classes, predicted_classes, average='weighted')

# In ra kết quả
print('Accuracy:', accuracy * 100)
print('Precision:', precision * 100)
print('Recall:', recall * 100)
print('F1 Score:', f1* 100)

# In báo cáo chi tiết
print('\nClassification Report:')
print(classification_report(true_classes, predicted_classes, target_names=class_names))

# Tính accuracy cho từng nhãn
conf_matrix = confusion_matrix(true_classes, predicted_classes)
class_accuracies = conf_matrix.diagonal() / conf_matrix.sum(axis=1)

print('\nAccuracy for each class:')
for class_name, class_accuracy in zip(class_names, class_accuracies):
    print(f'{class_name}: {class_accuracy:.2f}')

12/12 [==============================] - 24s 1s/step
Accuracy: 41.05263157894737
Precision: 43.79420941351629
Recall: 41.05263157894737
F1 Score: 35.35201597772743

Classification Report:
                    precision    recall  f1-score   support

 banh-hoi-heo-quay       0.56      0.60      0.58        15
        bun-bo-hue       0.28      0.58      0.38        36
            bun-ca       0.50      0.15      0.23        20
   bun-dau-mam-tom       0.50      0.92      0.65        25
 bun-hai-san-be-be       0.00      0.00      0.00        30
           bun-moc       0.80      0.21      0.33        19
           bun-muc       0.00      0.00      0.00        13
      bun-nuoc-leo       0.00      0.00      0.00        26
           cao-lau       0.50      0.38      0.43        37
            com-ga       0.74      0.50      0.60        40
com-tam-long-xuyen       0.56      0.69      0.62        36
    hu-tieu-my-tho       0.00      0.00      0.00        14
          mi-quang       1.00  

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:13

# Kết quả Xception

In [30]:
from sklearn.metrics import confusion_matrix

model = tf.keras.models.load_model('/content/drive/MyDrive/object_detection/models/xception.h5')

predictions = model.predict(test_generator)
predicted_classes = np.argmax(predictions, axis=1)

# Lấy nhãn thực tế từ generator
true_classes = test_generator.classes

# Tính các chỉ số đánh giá
accuracy = accuracy_score(true_classes, predicted_classes)
precision = precision_score(true_classes, predicted_classes, average='weighted')
recall = recall_score(true_classes, predicted_classes, average='weighted')
f1 = f1_score(true_classes, predicted_classes, average='weighted')

# In ra kết quả
print('Accuracy:', accuracy * 100)
print('Precision:', precision * 100)
print('Recall:', recall * 100)
print('F1 Score:', f1* 100)

# In báo cáo chi tiết
print('\nClassification Report:')
print(classification_report(true_classes, predicted_classes, target_names=class_names))

# Tính accuracy cho từng nhãn
conf_matrix = confusion_matrix(true_classes, predicted_classes)
class_accuracies = conf_matrix.diagonal() / conf_matrix.sum(axis=1)

print('\nAccuracy for each class:')
for class_name, class_accuracy in zip(class_names, class_accuracies):
    print(f'{class_name}: {class_accuracy:.2f}')

12/12 [==============================] - 12s 505ms/step
Accuracy: 52.10526315789473
Precision: 67.46538003996166
Recall: 52.10526315789473
F1 Score: 49.26466827339202

Classification Report:
                    precision    recall  f1-score   support

 banh-hoi-heo-quay       0.65      0.87      0.74        15
        bun-bo-hue       0.86      0.17      0.28        36
            bun-ca       0.60      0.15      0.24        20
   bun-dau-mam-tom       1.00      0.80      0.89        25
 bun-hai-san-be-be       0.88      0.47      0.61        30
           bun-moc       1.00      0.16      0.27        19
           bun-muc       0.67      0.31      0.42        13
      bun-nuoc-leo       1.00      0.27      0.42        26
           cao-lau       0.53      0.70      0.60        37
            com-ga       0.35      0.88      0.50        40
com-tam-long-xuyen       0.86      0.33      0.48        36
    hu-tieu-my-tho       0.00      0.00      0.00        14
          mi-quang       0.6

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:13

# Kết quả DenseNet121

In [31]:
from sklearn.metrics import confusion_matrix

model = tf.keras.models.load_model('/content/drive/MyDrive/object_detection/models/densenet121.h5')

predictions = model.predict(test_generator)
predicted_classes = np.argmax(predictions, axis=1)

# Lấy nhãn thực tế từ generator
true_classes = test_generator.classes

# Tính các chỉ số đánh giá
accuracy = accuracy_score(true_classes, predicted_classes)
precision = precision_score(true_classes, predicted_classes, average='weighted')
recall = recall_score(true_classes, predicted_classes, average='weighted')
f1 = f1_score(true_classes, predicted_classes, average='weighted')

# In ra kết quả
print('Accuracy:', accuracy * 100)
print('Precision:', precision * 100)
print('Recall:', recall * 100)
print('F1 Score:', f1* 100)

# In báo cáo chi tiết
print('\nClassification Report:')
print(classification_report(true_classes, predicted_classes, target_names=class_names))

# Tính accuracy cho từng nhãn
conf_matrix = confusion_matrix(true_classes, predicted_classes)
class_accuracies = conf_matrix.diagonal() / conf_matrix.sum(axis=1)

print('\nAccuracy for each class:')
for class_name, class_accuracy in zip(class_names, class_accuracies):
    print(f'{class_name}: {class_accuracy:.2f}')

12/12 [==============================] - 7s 364ms/step
Accuracy: 51.31578947368421
Precision: 67.06676150339067
Recall: 51.31578947368421
F1 Score: 46.65441021746036

Classification Report:
                    precision    recall  f1-score   support

 banh-hoi-heo-quay       0.82      0.60      0.69        15
        bun-bo-hue       1.00      0.03      0.05        36
            bun-ca       0.50      0.25      0.33        20
   bun-dau-mam-tom       0.81      1.00      0.89        25
 bun-hai-san-be-be       0.86      0.40      0.55        30
           bun-moc       0.29      0.89      0.44        19
           bun-muc       0.36      0.38      0.37        13
      bun-nuoc-leo       1.00      0.04      0.07        26
           cao-lau       0.67      0.54      0.60        37
            com-ga       0.88      0.35      0.50        40
com-tam-long-xuyen       0.43      0.89      0.58        36
    hu-tieu-my-tho       0.00      0.00      0.00        14
          mi-quang       0.80

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:13